In [9]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import math

plt.rcParams["figure.figsize"] = (18, 9)

In [10]:
train_path = "andan-2022-autumn-lab-3/train.csv"
test_path = "andan-2022-autumn-lab-3/test_x.csv"

In [11]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [ ]:
train_rubrics_set = set()
for i in train_data['rubrics']:
    train_rubrics_set.add(i)

def rubrics_handler(s):
    return [0.5 if s == i else 0 for i in train_rubrics_set]

columns = train_data.columns
name_to_id = dict()
org_id = None
rt = None
rubrics = None
def line_handler(line):
    # global columns
    # global name_to_id
    ma = []
    for name in columns:
        # print(name, ma)
        if name in ['rating_org',
                    'user_id',
                    'average_bill',
                    'ts',
                    'org_id',
                    ]: # ignore
            continue

        name_to_id[name] = len(ma)
        if False: pass
        elif name in ['rating']:
            ma += [int(line[name])]
        elif name in ['rubrics']:
            ma += rubrics_handler(line[name])
        elif name in ['org_city', 'user_city']:
            ma += [0 if line[name] == 'msk' else 1]
        else:
            ma += [line[name]]
    global org_id, rubrics, rt
    # org_id = name_to_id['org_id']
    rubrics = name_to_id['rubrics']
    rt = name_to_id['rating']
    return ma

def metric(ma1, ma2):
    dist = 0
    global org_id, rubrics, rt
    for i in range(len(ma1)):
        if ma1[i] != ma2[i]:
            if org_id == i:
                dist += 100
            if rt == i:
                dist += abs(ma1[i] - ma2[i])
            elif i in range(rubrics, rubrics + len(train_rubrics_set)):
                dist += 0.5
            else:
                dist += 1
    return dist

def getRegression(data):
    X = []
    # columns = train_data.columns
    for i in data.index:
        line = data.iloc[i]
        X += [line_handler(line)]
        # print(X)

    print(X[0])
    X = np.array(X)
    print(X[0])
    # neigh = KNeighborsRegressor(n_neighbors=5, metric=metric, n_jobs=-1)
    # neigh.fit(X, rating_org)
    # return neigh
    return X

#reg = getRegression(train_data, train_data['rating_org'])
X, rating_org = getRegression(train_data), train_data['rating_org']

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
# 1) n_neighbors=5, weights='uniform', p=2
# 2) n_neighbors=int(len(train_data)**0.5), weights='uniform', p=2
# 3) n_neighbors=int(len(train_data)**0.5), weights='uniform', p=2
# 4) n_neighbors=int(len(train_data)**0.5), weights='uniform', p=1
### BEST SCORE FOR FIRST 4 PUSHES
neigh = KNeighborsRegressor(n_neighbors=int(len(train_data)**0.5),
                            weights='distance',
                            # metric=metric,
                            p=1,
                            n_jobs=-1,
                            )
neigh.fit(X, rating_org)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
prc = neigh.predict([line_handler(train_data.iloc[i]) for i in train_data.index])
mean_absolute_percentage_error(train_data["rating_org"], prc)

In [30]:
ma = neigh.predict([line_handler(train_data.iloc[i]) for i in test_data.index])
# ma = neigh.predict([line_handler(train_data.iloc[i]) for i in test_data.index[:100]])

rating_pred = pd.DataFrame(np.array(ma), columns=['rating_org'])

In [ ]:
from itertools import count
rating_pred = pd.DataFrame(zip(count(0), np.array(ma)), columns=['id', 'rating_org'])
rating_pred.to_csv('andan-2022-autumn-lab-3/saved_ratings.csv', index=False)
print(rating_pred)